In [1]:
import numpy as np
import pandas as pd
import pickle
from pathlib import Path
from hloc.utils.read_write_model import read_images_binary

# Configure paths
IMAGES_PATH = '../outputs/aachen/sfm_superpoint+superglue/images.bin'
OUTPUT_DIR = Path('../5FoldDataset')
OUTPUT_DIR.mkdir(exist_ok=True)

# Load images
images = read_images_binary(IMAGES_PATH)

In [2]:
def load_database():
    """Load the main database and validation datasets"""
    with open('../DataBase/DataBase_Norm_Tiny.pickle', 'rb') as f:
        return pickle.load(f)

database = load_database()

In [3]:
def create_image_path(key):
    """Convert image key to path"""
    return images[key].name

def process_validation_set(val_data):
    """Process validation dataset to extract anchor images"""
    anchor_values = []
    for item in val_data:
        anchor_values.append(item['Anchor'][0])
    
    return {'Anchor': [create_image_path(val) for val in anchor_values]}

def get_ordinal_suffix(fold_number):
    """Return the correct ordinal suffix for a number"""
    if fold_number == 1:
        return "st"
    elif fold_number == 2:
        return "nd"
    elif fold_number == 3:
        return "rd"
    else:
        return "th"

def save_to_csv(data, fold_number):
    """Save processed data to CSV file"""
    suffix = get_ordinal_suffix(fold_number)
    df = pd.DataFrame(data)
    output_path = OUTPUT_DIR / f'anchor_student_{fold_number}{suffix}.csv'
    df.to_csv(output_path, index=False)
    print(f"Saved {output_path} with {len(df)} entries")

In [4]:
def process_all_folds():
    """Process all 5 folds of validation data"""
    # Map fold numbers to their corresponding file names
    fold_files = {
        1: 'First_Val_Database_5.pkl',
        2: 'Second_Val_Database_4.pkl',
        3: 'Third_Val_Database_3.pkl',
        4: 'Fourth_Val_Database_2.pkl',
        5: 'Fifth_Val_Database_1.pkl'
    }
    
    for fold_num, file_name in fold_files.items():
        # Load validation data
        val_path = OUTPUT_DIR / file_name
        try:
            with open(val_path, 'rb') as f:
                val_data = pickle.load(f)
            
            # Process and save
            anchor_data = process_validation_set(val_data)
            save_to_csv(anchor_data, fold_num)
            
        except FileNotFoundError:
            print(f"Warning: Could not find validation file for fold {fold_num}: {val_path}")

# Execute processing
process_all_folds()

Saved ../5FoldDataset/anchor_student_1st.csv with 868 entries
Saved ../5FoldDataset/anchor_student_2nd.csv with 864 entries
Saved ../5FoldDataset/anchor_student_3rd.csv with 864 entries
Saved ../5FoldDataset/anchor_student_4th.csv with 864 entries
Saved ../5FoldDataset/anchor_student_5th.csv with 864 entries


In [5]:
def verify_outputs():
    """Verify that all expected files were created and show their sizes"""
    # 올바른 서수 접미사를 사용하여 파일 이름 생성
    expected_files = [f'anchor_student_{i}{get_ordinal_suffix(i)}.csv' for i in range(1, 6)]
    
    print("Verification Results:")
    print("-" * 50)
    for file_name in expected_files:
        file_path = OUTPUT_DIR / file_name
        if file_path.exists():
            df = pd.read_csv(file_path)
            print(f"{file_name}: {len(df)} entries")
        else:
            print(f"{file_name}: File not found")
    print("-" * 50)

verify_outputs()

Verification Results:
--------------------------------------------------
anchor_student_1st.csv: 868 entries
anchor_student_2nd.csv: 864 entries
anchor_student_3rd.csv: 864 entries
anchor_student_4th.csv: 864 entries
anchor_student_5th.csv: 864 entries
--------------------------------------------------
